# 项目分析：
- 构成：
    - 蛇 snake
    
    - 食物 food
    
    - 边界 World
    
- 蛇和食物属于整个世界
            
            class World:
                self.snake
                self.food
     - 上面代码不太友好
     - 我们换个思路
     
- 我们的思路
    - 食物是一个独立的食物
    - 蛇也一个独立的食物
    - 世界就是个背景。它只规定了边界

In [26]:
import random
import tkinter
import threading

class Food():
    '''
    出现在世界的一个地方
    一旦被吃蛇加分，它重新刷新一个出来。
    '''
    def __init__(self, queue):
        '''
        生成一个食物
        '''
        self.queue = queue
        self.new_food()
    
    def new_food(self):
        '''
        功能，new一个食物
        产生一个食物的过程，实际上就是生成一个坐标
        '''
        # 创建食物坐标
        x = random.randrange(30,450,10)
        y = random.randrange(30,450,10)
        
        self.position = x,y # 存放食物的位置
        
        # 队列，就是一个不能够随意访问内部元素，只能从头弹出一个元素。并只能从队尾追加元素的列表。其实就像排队买票一样
        # 把一个食物产生的消息放入队列
        # 消息的格式是自己定义的
        # 我的定义是：消息是一个dict，k代表消息类型，v代表此类型的数据
        
        self.queue.put({'food':self.position})


In [27]:
class Snake(threading.Thread):
    '''
    蛇的功能：
        1 会动，要move()，由方向键控制
        2 蛇每次移动都要重新计算蛇头的位置，身子还要跟着前者动。
        3 检测游戏是否完事儿了，好结束运行
    '''
    def __init__(self, world, queue):
        threading.Thread.__init__(self)
        
        self.world = world
        self.queue
        self.points_earned = 0 #fenshu
        self.food = Food(self.queue)
        self.snake_pos = [(495, 55), (485, 55), (475, 55), (465, 55), (455, 55), (445, 55)]
        
        self.start()
        
    def run(self):
        '''一旦启用多线程调用此函数，要求蛇一直在跑'''
        if self.world.is_game_over:
            self._delete()
        while not self.world.is_game_over:
            self.queue.put({"move":self.snake_pos})
            time.sleep(0.5) #控制蛇的速度
            self.move() 
    
    def move(self):
        '''
        负责蛇的移动
        1 重新计算蛇头坐标
        2 蛇头和食物相遇，则加分，加长。通知world加分。
        3 否则蛇继续动
        '''
        
        new_snake_pos = self.cal_new_pos() # 重新计算蛇头的位置
        
        # 吃到食物
        if self.food.position == new_snake_pos:
            self.points_earnd += 1 # 得分加1
            self.queue.put({'points_earned':self.points_earned})
            self.food.new_food() # 生成新的食物
        else:
            # 要注意蛇的信息的保存方式
            # 每次移动是删除存放snake位置信息的序列的第一个变量，并追加在后面，因为序列是从尾部到头部保存的。
            self.snake_pos.pop(0)
            # 判断程序是否退出，因为新的蛇可能撞墙
            self.check_game_over(new_snake_pos)
            self.snake_pos.append(new_snake_pos)
    
    def cal_new_pos(self):
        '''计算新蛇头位置'''
        
        last_x,last_y = self.snake_pos[-1]
        if self.direction == 'Up': # derection负责储存蛇的移动方向
            new_snake_pos = last_x,last_y + 10 # 每步10像素
        if self.direction == 'Down': 
            new_snake_pos = last_x,last_y - 10    
        if self.direction == 'Right': 
            new_snake_pos = last_x + 10,last_y    
        if self.direction == 'Left': 
            new_snake_pos = last_x - 10,last_y    
        
        return new_snake_pos
            
    def key_pressed(self,e):
        # keysym是按键名称
        self.direction = e.keysym2
        
        
    def check_game_over(self, snake_loc):
        '''判断蛇头是否撞墙'''
        x,y = snake_loc[0],snake_loc[1]
        if x <= 0 or x >= 640 or y <= 0 or y >= 480 or snake_loc in self.snake_pos:
            self.queue.put({'game_over':True})


In [33]:
import queue

class World():
    '''用来描述整个游戏画板'''
    
    def __init__(self, queue):

        self.queue = queue
        self.queue.put({'move':None})
        self.is_game_over = False
        self.root = tkinter.Tk()
        
        # 定义画板
        self.canvas = tkinter.Canvas(self.root, width=640, height=480, bg='red')
        self.canvas.pack()
        
        # 画蛇和食物
        self.snake = self.canvas.create_line((0,0), (0,0), fill="black", width=10)
        self.food = self.canvas.create_rectangle(0,0,0,0, fill="blue", outline='blue')
        
        self.point_earned = self.canvas.create_text(600,400)
        self.queue_handler()
        
    def queue_handler(self):
        # 需要不断从消息队列取回消息，所以死循环即可
        while True:
            try:
                task = self.queue.get(block=False)

                if task.get("game_over"):
                    self.game_over()
                elif task.get("move"):
                    points = [x for point in task['move'] for x in point]
                    #  重新绘制蛇
                    self.canvas.coords(self.snake, *point)
                #　同样道理处理食物和得分
                elif task.get('food'):
                    self.new_food()
#                 elif task.get('points_earned'):
#                     self.
                
                
                
            except queue.Empty: # 爆出队列为空异常
                if not self.is_game_over:
                    self.canvas.after(100, self.queue_handler)
                    
    def game_over(self):
        '''
        游戏结束，清理现场
        '''
        self.is_game_over = True
        self.canvas.create_text('Game Over')
        qb = Button(self, text='Quit', command=self.destroy)
        rb = Button(self, text="Again", command=self.__init__)
        
if __name__ == "__main__":
    q = queue.Queue()
    world = World(q)
    food = Food(q)
    snake = Snake(world,q)
    
    # 绑定按钮
    world.bind('<Key-Left>', snake.key_pressed)
    world.bind('<Key-Right>', snake.key_pressed)
    world.bind('<Key-Up>', snake.key_pressed)
    world.bind('<Key-Down>', snake.key_pressed)
    
    world.mainloop()
    

KeyboardInterrupt: 